## 1、ChatMessageHistory(基础)的使用

In [1]:
from langchain.chains.summarize.map_reduce_prompt import prompt_template
# 场景一：记忆存储
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import PromptTemplate

# 1、ChatMessageHistory实例化
history = ChatMessageHistory()

# 2、添加消息
history.add_user_message("你好")
history.add_ai_message("你好！有什么我可以帮助你的吗？")

# 3、获取消息
messages = history.messages
for message in messages:
	print(f"{message.type}: {message.content}")

human: 你好
ai: 你好！有什么我可以帮助你的吗？


In [3]:
# 场景二：与LLM集成
from langchain.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory
import os
import dotenv

# 1、加载环境变量
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
# 2、实例化ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# 3、实例化ChatMessageHistory
history = ChatMessageHistory()
# 4、添加用户消息
history.add_user_message("你好")
# 5、获取消息并调用LLM
response = llm.invoke(history.messages)
print("AI:", response.content)
# 6、添加AI消息到历史
history.add_ai_message(response.content)
# 7、继续对话
history.add_user_message("今天天气怎么样？")
response = llm.invoke(history.messages)
print("AI:", response.content)
history.add_ai_message(response.content)
# 8、查看完整对话历史
for message in history.messages:
	print(f"{message.type}: {message.content}")

AI: 你好！有什么我可以帮助你的吗？
AI: 我无法实时获取天气信息，但你可以通过天气应用、网站或当地新闻来查看今天天气。如果你告诉我你所在的城市，我可以给你一些建议，如何查找天气信息！
human: 你好
ai: 你好！有什么我可以帮助你的吗？
human: 今天天气怎么样？
ai: 我无法实时获取天气信息，但你可以通过天气应用、网站或当地新闻来查看今天天气。如果你告诉我你所在的城市，我可以给你一些建议，如何查找天气信息！


## 2、ConversationBufferMemory的使用
适用场景：对话轮次较少、依赖完整上下文的场景（如简单的聊天机器）

In [3]:
# 举例1：以字符串的方式返回存储的信息
from langchain.memory import ConversationBufferMemory

# 1、ConversationBufferMemory()的实例化
memory = ConversationBufferMemory()

# 2、存储相关的信息
# 注意：inputs对应的是用户输入，outputs对应的是AI的回复
memory.save_context(
	inputs={
		"input": "你好"
	},
	outputs={
		"output": "你好！有什么我可以帮助你的吗？"
	}
)
memory.save_context(
	inputs={
		"input": "请问1.8何1.11谁大？"
	},
	outputs={
		"output": "1.8比较大"
	}
)

# 3、获取存储的信息
print(memory.load_memory_variables({}))

# 返回的字典结构的key叫做history，value是一个字符串

{'history': 'Human: 你好\nAI: 你好！有什么我可以帮助你的吗？\nHuman: 请问1.8何1.11谁大？\nAI: 1.8比较大'}


In [5]:
# 举例2：以消息列表的方式返回存储的信息
from langchain.memory import ConversationBufferMemory

# 1、ConversationBufferMemory()的实例化
memory = ConversationBufferMemory(return_messages=True)

# 2、存储相关的信息
# 注意：inputs对应的是用户输入，outputs对应的是AI的回复
memory.save_context(
	inputs={
		"input": "你好"
	},
	outputs={
		"output": "你好！有什么我可以帮助你的吗？"
	}
)
memory.save_context(
	inputs={
		"input": "请问1.8何1.11谁大？"
	},
	outputs={
		"output": "1.8比较大"
	}
)

# 3、获取存储的信息
# 返回消息列表的方式1
print(memory.load_memory_variables({}))
print()
# 返回消息列表的方式2
print(memory.chat_memory.messages)

{'history': [HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='你好！有什么我可以帮助你的吗？', additional_kwargs={}, response_metadata={}), HumanMessage(content='请问1.8何1.11谁大？', additional_kwargs={}, response_metadata={}), AIMessage(content='1.8比较大', additional_kwargs={}, response_metadata={})]}

[HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='你好！有什么我可以帮助你的吗？', additional_kwargs={}, response_metadata={}), HumanMessage(content='请问1.8何1.11谁大？', additional_kwargs={}, response_metadata={}), AIMessage(content='1.8比较大', additional_kwargs={}, response_metadata={})]


In [10]:
# 举例3: 与LLM、提示词模板（PromptTemplate）集成
import os
import dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain

# 1、加载环境变量
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

# 2、实例化ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3、提供提示词模板
prompt_template = PromptTemplate.from_template(
	template="""你可以与人类对话。

当前对话历史: {history}

人类问题: {question}

回复:
	"""
)

# 4、实例化ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True)

# 5、提供Chain
chain = LLMChain(
	llm=llm,
	prompt=prompt_template,
	memory=memory
)

# 6、进行对话
response = chain.invoke({"question": "你好,我的名字叫小明"})
# print(response)

response = chain.invoke({"question": "我叫什么名字？"})
print(response)

{'question': '我叫什么名字？', 'history': [HumanMessage(content='你好,我的名字叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='你好，小明！很高兴认识你。有什么我可以帮助你的吗？', additional_kwargs={}, response_metadata={}), HumanMessage(content='我叫什么名字？', additional_kwargs={}, response_metadata={}), AIMessage(content='你叫小明。', additional_kwargs={}, response_metadata={})], 'text': '你叫小明。'}


In [11]:
# 举例4:基于举例3显示的设置memory的key的值
import os
import dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain

# 1、加载环境变量
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

# 2、实例化ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3、提供提示词模板
prompt_template = PromptTemplate.from_template(
	template="""你可以与人类对话。

当前对话历史: {chat_history}

人类问题: {question}

回复:
	"""
)

# 4、实例化ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")

# 5、提供Chain
chain = LLMChain(
	llm=llm,
	prompt=prompt_template,
	memory=memory
)

# 6、进行对话
response = chain.invoke({"question": "你好,我的名字叫小明"})
# print(response)

response = chain.invoke({"question": "我叫什么名字？"})
print(response)

{'question': '我叫什么名字？', 'chat_history': 'Human: 你好,我的名字叫小明\nAI: 你好，小明！很高兴认识你。有什么我可以帮助你的吗？', 'text': '你叫小明。'}


In [12]:
# 举例5：结合大模型、提示词模板（ChatPromptTemplate）的使用
# 1.导入相关包
from langchain_core.messages import SystemMessage
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI

# 2.创建LLM
llm = ChatOpenAI(model_name='gpt-4o-mini')
# 3.创建Prompt
prompt = ChatPromptTemplate.from_messages([
	("system", "你是一个与人类对话的机器人。"),
	MessagesPlaceholder(variable_name='history'),
	("human", "问题：{question}")
])
# 4.创建Memory
memory = ConversationBufferMemory(return_messages=True)
# 5.创建LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)
# 6.调用LLMChain
res1 = llm_chain.invoke({"question": "中国首都在哪里？"})
print(res1, end="\n\n")
res2 = llm_chain.invoke({"question": "我刚刚问了什么"})
print(res2)

{'question': '中国首都在哪里？', 'history': [HumanMessage(content='中国首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国的首都位于北京。', additional_kwargs={}, response_metadata={})], 'text': '中国的首都位于北京。'}

{'question': '我刚刚问了什么', 'history': [HumanMessage(content='中国首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国的首都位于北京。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我刚刚问了什么', additional_kwargs={}, response_metadata={}), AIMessage(content='你刚刚问了中国首都在哪里。', additional_kwargs={}, response_metadata={})], 'text': '你刚刚问了中国首都在哪里。'}


| 特性 | 普通 PromptTemplate           | ChatPromptTemplate |
|:----:|:---------------------------:|:-----------------:|
| 历史存储时机 | 仅执行后存储                      | 执行前存储用户输入 + 执行后存储输出 |
| 首次调用显示 | 仅显示问题（历史仍为空字符串）             | 显示完整问答对 |
| 内部消息类型 | 拼接字符串                       | List[BaseMessage] |

## 3、ConversationChain
ConversationChain实际上是就是对 ConversationBufferMemory 和 LLMChain 进行了封装，并且提供一个默认格式的提示词模版（我们也可以不用），从而简化了初始化ConversationBufferMemory的步骤。


In [15]:
# 举例1
import os
import dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain

# 1、加载环境变量
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

# 2、实例化ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3、提供提示词模板 - 修正：不要使用f-string
prompt_template = PromptTemplate.from_template(
	template="""你可以与人类对话。

当前对话历史: {history}

人类问题: {input}

回复:"""
)

# 4、创建ConversationChain
chain = ConversationChain(
	llm=llm,
	prompt=prompt_template,
	# memory=memory # 如果不传memory参数，则默认使用ConversationBufferMemory
)

# 5、进行对话
response = chain.invoke({"input": "你好,我的名字叫小明"})
print(response)

response = chain.invoke({"input": "我叫什么名字？"})
print(response)

{'input': '你好,我的名字叫小明', 'history': '', 'response': '你好，小明！很高兴认识你。有什么我可以帮助你的吗？'}
{'input': '我叫什么名字？', 'history': 'Human: 你好,我的名字叫小明\nAI: 你好，小明！很高兴认识你。有什么我可以帮助你的吗？', 'response': '你叫小明。你还有其他想聊的吗？'}


In [16]:
# 举例2：使用默认的提示词模板
import os
import dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

# 1、加载环境变量
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

# 2、实例化ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3、创建ConversationChain
# 内部提供了默认的提示词模板，此模板中的变量是「input」、「history」
chain = ConversationChain(
	llm=llm,
	# memory=memory # 如果不传memory参数，则默认使用ConversationBufferMemory
)

# 4、进行对话
response = chain.invoke({"input": "你好,我的名字叫小明"})
print(response)

response = chain.invoke({"input": "我叫什么名字？"})
print(response)

{'input': '你好,我的名字叫小明', 'history': '', 'response': '你好，小明！很高兴认识你！我叫AI助手。你今天过得怎么样？有什么特别的事情想分享吗？'}
{'input': '我叫什么名字？', 'history': 'Human: 你好,我的名字叫小明\nAI: 你好，小明！很高兴认识你！我叫AI助手。你今天过得怎么样？有什么特别的事情想分享吗？', 'response': '你叫小明！我记得你刚刚告诉我你的名字。你还有其他想聊的事情吗？'}


## 4、ConversationBufferWindowMemory
在了解了ConversationBufferMemory记忆类后，我们知道了它能够无限的将历史对话信息填充到History中，从而给大模型提供上下文的背景。但这会导致内存量十分大 ，并且消耗的token是非常多的，此外，每个大模型都存在最大输入的Token限制。

我们发现，过久远的对话数据往往并不能对当前轮次的问答提供有效的信息，LangChain 给出的解决方式是： ConversationBufferWindowMemory 模块。该记忆类会 保存一段时间内对话交互 的列表， 仅使用最近 K 个交互 。这样就使缓存区不会变得太大。

In [17]:
# 举例1
# 1.导入相关包
from langchain.memory import ConversationBufferWindowMemory

# 2.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=2)
# 3.保存消息
memory.save_context({"input": "你好"}, {"output": "怎么了"})
memory.save_context({"input": "你是谁"}, {"output": "我是AI助手"})
memory.save_context({"input": "你的生日是哪天？"}, {"output": "我不清楚"})
# 4.读取内存中消息（返回消息内容的纯文本）
print(memory.load_memory_variables({}))

{'history': 'Human: 你是谁\nAI: 我是AI助手\nHuman: 你的生日是哪天？\nAI: 我不清楚'}


C:\Users\Chenxr\AppData\Local\Temp\ipykernel_20220\2753770296.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2)


In [18]:
# 举例2
# 1.导入相关包
from langchain.memory import ConversationBufferWindowMemory

# 2.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=2, return_messages=True)
# 3.保存消息
memory.save_context({"input": "你好"}, {"output": "怎么了"})
memory.save_context({"input": "你是谁"}, {"output": "我是AI助手小智"})
memory.save_context({"input": "初次对话，你能介绍一下你自己吗？"}, {"output": "当然可以了。我是一个无所不能的小智。"})
# 4.读取内存中消息（返回消息内容的纯文本）
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='你是谁', additional_kwargs={}, response_metadata={}), AIMessage(content='我是AI助手小智', additional_kwargs={}, response_metadata={}), HumanMessage(content='初次对话，你能介绍一下你自己吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='当然可以了。我是一个无所不能的小智。', additional_kwargs={}, response_metadata={})]}


In [21]:
# 举例3：结合llm、chain
from langchain.memory import ConversationBufferWindowMemory
# 1.导入相关包
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain

# 2.定义模版
template = """以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：
{history}
Human: {question}
AI:"""
# 3.定义提示词模版
prompt_template = PromptTemplate.from_template(template)
# 4.创建大模型
llm = ChatOpenAI(model="gpt-4o-mini")
# 5.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=1)
# 6.定义LLMChain
conversation_with_summary = LLMChain(
	llm=llm,
	prompt=prompt_template,
	memory=memory,
	verbose=True,
)
# 7.执行链（第一次提问）
respon1 = conversation_with_summary.invoke({"question": "你好，我是孙小空"})
# print(respon1)
# 8.执行链（第二次提问）
respon2 = conversation_with_summary.invoke(
	{"question": "我还有两个师弟，一个是猪小戒，一个是沙小僧"})
# print(respon2)
# 9.执行链（第三次提问）
respon3 = conversation_with_summary.invoke(
	{"question": "我今年高考，竟然考上了1本"})
# print(respon3)
# 10.执行链（第四次提问）
respon4 = conversation_with_summary.invoke({"question": "我叫什么？"})
print(respon4)



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：

Human: 你好，我是孙小空
AI:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：
Human: 你好，我是孙小空
AI: 你好，孙小空！很高兴认识你！你今天过得怎么样？有什么想聊的话题吗？
Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧
AI:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：
Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧
AI: 听起来你和你的师弟们都有很有趣的名字！猪小戒和沙小僧分别代表了《西游记》中的角色吗？你们平时一起做些什么活动呢？
Human: 我今年高考，竟然考上了1本
AI:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：
Human: 我今年高考，竟然考上了1本
AI: 恭喜你考上了1本！这真是一个了不起的成就！你一定付出了很多努力。你打算在哪个专业学习呢？或者你有没有心仪的大学？
Human: 我叫什么？
AI:

> Finished chain.
{'question': '我叫什么？', 'history': 'Human: 我今年高考，竟然考上了1本\nAI: 恭喜你考上了1本！这